In [1]:
import os
import csv
from scapy.all import rdpcap, IP, TCP, UDP

In [2]:
def get_flow_key(pkt):
    if IP in pkt:
        proto = pkt[IP].proto
        ip_src = pkt[IP].src
        ip_dst = pkt[IP].dst

        if proto == 6 and TCP in pkt:
            sport = pkt[TCP].sport
            dport = pkt[TCP].dport
        elif proto == 17 and UDP in pkt:
            sport = pkt[UDP].sport
            dport = pkt[UDP].dport
        else:
            return None  # ignora se não for TCP ou UDP

        return (ip_src, sport, ip_dst, dport, proto)
    return None

def process_pcap_to_csv(pcap_path, output_dir):
    packets = rdpcap(pcap_path)

    flows = {}  # agrupar por 5-tupla

    for pkt in packets:
        key = get_flow_key(pkt)
        if not key:
            continue
        if key not in flows:
            flows[key] = []
        flows[key].append(pkt)

    os.makedirs(output_dir, exist_ok=True)
    for i, (key, pkts) in enumerate(flows.items()):
        ip_src, sport, ip_dst, dport, proto = key
        flow_id = f"flow_{i}"
        start_time = pkts[0].time
        times = [round(pkt.time - start_time, 6) for pkt in pkts]
        sizes = [len(pkt) for pkt in pkts]

        row = [
            flow_id,
            ip_src,
            sport,
            ip_dst,
            dport,
            proto,
            0,  # id1 (placeholder)
            0,  # id2 (placeholder)
            0.0  # tempo relativo inicial (sempre 0)
        ] + times[1:] + [''] + sizes  # separar tempo e tamanho com ''

        csv_name = os.path.join(output_dir, f"{flow_id}.csv")
        with open(csv_name, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row)

        print(f"[✓] Exported {csv_name} with {len(pkts)} packets.")

In [6]:
# Exemplo de uso:
process_pcap_to_csv('../data/ISCX-VPN-nonVPN-2016/VPN-PCAPS-01/vpn_hangouts_audio1.pcap', "../Classes_csv/VOIP/VPN")


[✓] Exported ../Classes_csv/VOIP/VPN\flow_0.csv with 6 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_1.csv with 8 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_2.csv with 6 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_3.csv with 8 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_4.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_5.csv with 6091 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_6.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_7.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_8.csv with 7234 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_9.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_10.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_11.csv with 1 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_12.csv with 8313 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_13.csv with 9642 packets.
[✓] Exported ../Classes_csv/VOIP/VPN\flow_14.csv with 15 packets.
[✓] Exported ../Classe

# Próximos Passos

1. rode esse notebook nos arquivos .pcap que preferir (isso vai transformar eles em arquivos .csv contendo os fluxos de transferencia de packets) É importante lembrar que é neecssário criar uma pasta para cada tipo de app e tipo de criptografia, e.g., Chat/VPN.
2. rode o Filter_data.py com o Labels.csv dentro da pasta Classes_csv (isso vai filtrar, reorganizar e gerar os histogramas), eles serão salvos dentro da pasta "ISCX-VPN-nonVPN-2016", vc pode mudar isso pra deixar mais organizado. Obs.: Remova o Labels.csv quando terminar de rodar, para n dar conflito com o prox passo.  importante lembrar que é neecssário criar uma pasta para cada tipo de app na pasta Pictures, e.g., Chat/VPN.
3. rode Repartition.py para dividir os dados em trein/valid/test
4. rode Main_SNN-TOR-VPN.py para criar o modelo.


dicas para melhorar o job:

1. melhore esse notebook de modo que ele posso executar o processamento em todos os .pcap de uma só vez (quissá até nos demais arquivos para gerar um só dataset com os dois disponiveis)
2. conserte essa questão do Labels.csv, deixe fixa a sua posição.
3. renomeie data para algo como Dataset_raw para manter o padrão com o restante do repo
4. reveja a necessidade dos histogramas terem que ser armazenados dentro "ISCX-VPN-nonVPN-2016", n seria mlr ter uma pasta "histos"?
5. limpar os notebooks deixando o que eh antigo em 'archive' e transformar esse notebbok em uma classe python
6. renomear os arquivos para snake_case para deixar mais normal?

